# RP3beta

## Import

In [1]:
## Allow more than one output for a single code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
import scipy.sparse as sps
import numpy as np
import os

from skopt.space import Real, Integer, Categorical

## Set the numpy random seed
SEED = 42
np.random.seed(SEED)

os.getcwd()

'/home/alessio/Scrivania/RecSysChallenge2021'

In [3]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

from Evaluation.Evaluator import EvaluatorHoldout

from Recommenders.Recommender_import_list import *

from Recommenders.DataIO import DataIO

Tensorflow is not available


In [4]:
## Utility Functions
from Dataset.load_data import load_data
from Dataset.write_submission import write_submission
from Dataset.load_test_user_array import load_test_user_array

## Data Loading and Split

In [5]:
URM_all, ICM_dict = load_data()

In [6]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

URM_aug_train = sps.vstack([URM_train.copy().tocoo(), 
                            #ICM_dict['ICM_genre'].T.tocoo(),
                            ICM_dict['ICM_subgenre'].T.tocoo(), 
                            #ICM_dict['ICM_event'].T.tocoo(), 
                            ICM_dict['ICM_channel'].T.tocoo()], format='csr')

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], exclude_seen = True)

EvaluatorHoldout: Ignoring 13646 ( 0.0%) Users that have less than 1 test interactions


In [7]:
test_UserID_array = load_test_user_array()

## Optimization

In [8]:
output_folder_path = "result_experiments/P3alpha_augmented_subgenre_channel/"

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(P3alphaRecommender.RECOMMENDER_NAME + "_metadata.zip")

#search_metadata.keys()

hyperparameters_df = search_metadata["hyperparameters_df"]
result_on_validation_df = search_metadata["result_on_validation_df"]

hyperparameters_df['MAP'] = result_on_validation_df['MAP'].values.tolist()

threshold = 0.20

hyperparameters_df[hyperparameters_df['MAP'] > threshold]
print('Max value for the range: ', max(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Min value for the range: ', min(hyperparameters_df[hyperparameters_df['MAP'] > threshold].topK))
print('Best MAP: ', max(hyperparameters_df['MAP']))
hyperparameters_df[max(hyperparameters_df['MAP']) == hyperparameters_df['MAP']]

,topK,alpha,normalize_similarity,MAP
28,126,0.765545,False,0.206943
38,202,0.731517,False,0.201909
61,85,0.711881,False,0.206780
96,115,0.740923,False,0.206167
97,281,0.809203,False,0.202563


Max value for the range:  281
Min value for the range:  85
Best MAP:  0.20694283050559711


,topK,alpha,normalize_similarity,MAP
28,126,0.765545,False,0.206943


In [9]:
output_folder_path = "result_experiments/P3alpha_augmented_subgenre_channel_improved_range/" # topK from 50 to 200

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 100  # 50 with 30% random is a good number
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [10]:
from functools import partial
import os, multiprocessing

from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative

runHyperparameterSearch_Collaborative(P3alphaRecommender,
                                      URM_train = URM_aug_train,
                                      URM_train_last_test = None,
                                      metric_to_optimize = metric_to_optimize,
                                      cutoff_to_optimize = cutoff_to_optimize,
                                      n_cases = n_cases,
                                      n_random_starts = n_random_starts,
                                      evaluator_validation_earlystopping = evaluator_validation,
                                      evaluator_validation = evaluator_validation,
                                      evaluator_test = None,
                                      output_folder_path = output_folder_path,
                                      resume_from_saved = True,
                                      similarity_type_list = None,
                                      parallelizeKNN = True)

SearchBayesianSkopt: Resuming 'P3alphaRecommender' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 161, 'alpha': 0.5468123789812366, 'normalize_similarity': False}
EvaluatorHoldout: Processed 13646 (100.0%) in 15.67 sec. Users per second: 871
SearchBayesianSkopt: New best config found. Config 0: {'topK': 161, 'alpha': 0.5468123789812366, 'normalize_similarity': False} - results: PRECISION: 0.3212883, PRECISION_RECALL_MIN_DEN: 0.3224677, RECALL: 0.0537929, MAP: 0.1944519, MAP_MIN_DEN: 0.1950933, MRR: 0.6087300, NDCG: 0.3450743, F1: 0.0921562, HIT_RATE: 0.9388099, ARHR_ALL_HITS: 1.0705606, NOVELTY: 0.0051896, AVERAGE_POPULARITY: 0.7245133, DIVERSITY_MEAN_INTER_LIST: 0.6882865, DIVERSITY_HERFINDAHL: 0.9688236, COVERAGE_ITEM: 0.0213190, COVERAGE_ITEM_CORRECT: 0.0100781, COVERAGE_USER: 0.9997070, COVERAGE_USER_CORRECT: 0.9385348, DIVERSITY_GINI: 0.0017884, SHANNON_ENTROPY: 5.3302468, RATIO_DIVERSITY_HE